In [12]:
import pandas as pd
import numpy as np 
import csv
import matplotlib.pyplot as plt
import math
from pandas_datareader import data
import scipy.optimize
from scipy import stats
from scipy.optimize import LinearConstraint
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.interpolate import griddata

In [14]:
#read tickers from csv file
tickers_file = 'TechTickers.csv'
tickers = [];
f = open(tickers_file, "r", encoding='utf-8-sig')
for line in csv.reader(f):
    tickers.append(str(line[0]))
f.close
#print(tickers)

<function TextIOWrapper.close()>

In [15]:
#download price data from Yahoo
start_date = '2009-01-01'
end_date = '2019-12-31'
stock_data = data.get_data_yahoo(tickers, start_date, end_date)

In [17]:
#create a list of tickers whose adjusted closing prices in the first month of the first year
#do not have any missing values
stockArray = []
for ticker in tickers:
    stockArray.append(list(stock_data['Adj Close'][ticker]))
stockArray=np.array(stockArray)
tickers_liq = []
for i in range(len(tickers)):
    temp=0
    for j in range(len(stockArray[i,:])):
        if math.isnan(stockArray[i,j]):
            temp=1
    if (temp==0):
        tickers_liq.append(tickers[i])
#print(len(tickers_liq))

64


In [8]:
#save the list of tickers without any initial missing values
tickers = list(tickers_liq) #use "list" when assigning a list to a new list in case mistakes
np.savetxt("TechTickers_liq.csv",np.array(tickers),fmt='%1s',delimiter=',')

In [9]:
#for the given range of years, download and save all adjusted closing prices for the tickers
#from the list of those without missing initial values
stockArray = []
for ticker in tickers_liq:
    stockArray.append(list(stock_data['Adj Close'][ticker]))
stockArray=np.array(stockArray)
np.savetxt("AdjustedClosingPrices_2009To2019.csv",np.transpose(stockArray),delimiter=',')

In [18]:
#some initialization

AnnualizedDay = 250; d = np.shape(stockArray)[0]
AdjPrice_file = "AdjustedClosingPrices_2009To2019.csv"
f = open(AdjPrice_file,"r",encoding='utf-8-sig')
AdjPrice = pd.read_csv('AdjustedClosingPrices_2009To2019.csv', header=None)
AdjPrice = np.array(AdjPrice)

#compute daily return
Ret = AdjPrice[1:,:] / AdjPrice[:-1,:] - 1

R0 = 0.01/AnnualizedDay #riskless asset daily return
gamma = 1/2 #parameter in utility function

#asset number
num_asset = 0

#trading settings
N = 250 #window size
T = 100 #trading time horizon

In [64]:
#(a) we construct a moving window and compute dynamic opt-strategy
d = 2
mod = (len(Ret[:,0]) - N) // 100
PnL1 = []
for i in range(mod+1):
    
    Calibri_Ret = Ret[i*T:i*T+N,num_asset]
    
    if i == mod:
        Trading_Ret = Ret[i*T+N:,num_asset]
    else:
        Trading_Ret = Ret[i*T+N:(i+1)*T+N,num_asset]
        
    mu = np.mean(Calibri_Ret)
    sigma = np.sqrt(np.var(Calibri_Ret))
    
    def my_obj1(x):
        return -((1-x)*R0 + x*mu + ((gamma-1)/2)*(x**2)*(sigma**2))
    
    opt1 = scipy.optimize.minimize_scalar(my_obj1)
    #opt_strategy1.append(opt1.x)
    #print(opt1.x,i)
    
    if i == 0:
        temp = 1
    else:
        temp = PnL1[-1]
    for j in range(len(Trading_Ret)):
        temp *= 1 + (1-opt1.x)*R0 + opt1.x*Trading_Ret[j]
        
    for j in range(len(Trading_Ret)):
        PnL1.append(temp)
    
#print(np.shape(PnL1))
#save data
np.savetxt("PnL_in_a.csv",PnL1,fmt='%1s',delimiter=',')

#output
print("Annualized mean and variance: ")
print(np.mean(PnL1)*AnnualizedDay, np.var(PnL1)*AnnualizedDay)
print('\n')
print("Sharpe Ratio is: ")
print(np.mean(PnL1)*AnnualizedDay / np.sqrt(np.var(PnL1)*AnnualizedDay))

Annualized mean and variance: 
1.740221402373899 0.06943642447261976


Sharpe Ratio is: 
6.60405727172277


In [75]:
#(b)adding predictive factors
VolumeArray = []
for ticker in tickers_liq:
    VolumeArray.append(list(stock_data['Volume'][ticker]))
VolumeArray=np.array(VolumeArray).T


rho = 1
Exp_WAve_Ret = []
Exp_Coeff = np.array([np.exp(-rho*i) for i in range(len(Ret[:,0]))])
for i in range(len(Ret[:,0])):
    Ret_Reverse = Ret[:,num_asset][i::-1]
    Vol_Reverse = VolumeArray[:,num_asset][i::-1]
    Exp_WAve_Ret.append(np.dot(Exp_Coeff[:i+1].T, np.multiply(Vol_Reverse, Ret_Reverse)))

slope, intercept, r_value, p_value, std_err = stats.linregress(Exp_WAve_Ret[:-1],Ret[1:,num_asset])

c = slope; a = intercept

In [90]:
#(b)
PnL2 = []
for i in range(mod+1):
    
    Calibri_Ret = Ret[i*T:i*T+N,num_asset]
    Calibri_Fac = Exp_WAve_Ret[i*T:i*T+N]
    Calibri_Vol = VolumeArray[i*T:i*T+N,num_asset]
    
    if i == mod:
        Trading_Ret = Ret[i*T+N:,num_asset]
        Trading_Fac = Exp_WAve_Ret[i*T+N:]
        Trading_Vol = VolumeArray[i*T+N:,num_asset]
    else:
        Trading_Ret = Ret[i*T+N:(i+1)*T+N,num_asset]
        Trading_Fac = Exp_WAve_Ret[i*T+N:(i+1)*T+N]
        Trading_Vol = VolumeArray[i*T+N:(i+1)*T+N,num_asset]
        
    mu = np.mean(Calibri_Ret)
    sigma = np.sqrt(np.var(Calibri_Ret))
    mu_hat = np.mean(Calibri_Vol)
    sigma_hat = np.sqrt(np.var(Calibri_Vol))
    mu_tilda = np.mean(Calibri_Fac)
    sigma_tilda = np.sqrt(np.var(Calibri_Fac))

    
    #the factor in the grid
    grid_f = [mu_tilda-3*sigma_tilda+k*0.01*6*sigma_tilda for k in range(100)]
    #grid_f = np.linspace(mu_tilda-3*sigma_tilda, mu_tilda+3*sigma_tilda,100)
    
    def value_v(x):
        return 1/gamma
    
    my_opt_strategy2 = []
    
    for t in range(len(Trading_Ret)-1,-1,-1):
        Interpol_Y = []; opt_stra_at_t = []
        for f in grid_f:
            def my_obj2(x):
                return -((1 + x*(a+c*f+sigma) + (1-x)*R0)**gamma * value_v(np.exp(-rho)*f + (mu_hat+sigma_hat)*(a+c*f+sigma))
                         + (1 + x*(a+c*f+sigma) + (1-x)*R0)**gamma * value_v(np.exp(-rho)*f + (mu_hat-sigma_hat)*(a+c*f+sigma))
                         + (1 + x*(a+c*f-sigma) + (1-x)*R0)**gamma * value_v(np.exp(-rho)*f + (mu_hat+sigma_hat)*(a+c*f-sigma))
                         + (1 + x*(a+c*f-sigma) + (1-x)*R0)**gamma * value_v(np.exp(-rho)*f + (mu_hat-sigma_hat)*(a+c*f-sigma))) / 4
            
            opt2 = scipy.optimize.minimize_scalar(my_obj2)
            
            Interpol_Y.append(((1 + opt2.x*(a+c*f+sigma) + (1-opt2.x)*R0)**gamma * value_v(np.exp(-rho)*f + (mu_hat+sigma_hat)*(a+c*f+sigma))
            + (1 + opt2.x*(a+c*f+sigma) + (1-opt2.x)*R0)**gamma * value_v(np.exp(-rho)*f + (mu_hat-sigma_hat)*(a+c*f+sigma))
            + (1 + opt2.x*(a+c*f-sigma) + (1-opt2.x)*R0)**gamma * value_v(np.exp(-rho)*f + (mu_hat+sigma_hat)*(a+c*f-sigma))
            + (1 + opt2.x*(a+c*f-sigma) + (1-opt2.x)*R0)**gamma * value_v(np.exp(-rho)*f + (mu_hat-sigma_hat)*(a+c*f-sigma))) / 4)
            
            opt_stra_at_t.append(opt2.x)    
        
        value_v = InterpolatedUnivariateSpline(grid_f, Interpol_Y, None, [grid_f[0], grid_f[-1]], 1)
        my_opt_strategy2.append(opt_stra_at_t)
        
    #print(my_opt_strategy2)
    my_opt_strategy2 = np.array(my_opt_strategy2)
    
    #now computing the PnL forwardly
    if i == 0:
        temp = 1
    else:
        temp = PnL2[-1]
    for j in range(len(Trading_Ret)):
        alpha2 = InterpolatedUnivariateSpline(grid_f, my_opt_strategy2[len(Trading_Ret)-j-1,:], None, [grid_f[0], grid_f[-1]], 1)
        opt_alpha2 = alpha2(Trading_Fac[j])
        temp *= 1 + (1-opt_alpha2)*R0 + opt_alpha2*Trading_Ret[j]
        
    for j in range(len(Trading_Ret)):
        PnL2.append(temp)
        
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [91]:
#(b) continuing
#save data
np.savetxt("PnL_in_b.csv",PnL2,fmt='%1s',delimiter=',')

#output
print("Annualized mean and variance: ")
print(np.mean(PnL2)*AnnualizedDay, np.var(PnL2)*AnnualizedDay)
print('\n')
print("Sharpe Ratio is: ")
print(np.mean(PnL2)*AnnualizedDay / np.sqrt(np.var(PnL2)*AnnualizedDay))

Annualized mean and variance: 
39.051502045438575 34.885695320810775


Sharpe Ratio is: 
6.611713876937684


In [111]:
#(c)
lambda_ = 0.01
PnL3 = []
for i in range(mod+1):
    
    Calibri_Ret = Ret[i*T:i*T+N,num_asset]
    
    if i == mod:
        Trading_Ret = Ret[i*T+N:,num_asset]
    else:
        Trading_Ret = Ret[i*T+N:(i+1)*T+N,num_asset]
        
    mu = np.mean(Calibri_Ret)
    sigma = np.sqrt(np.var(Calibri_Ret))
    
    
    #the factor in the grid
    grid_A = [-2+k*0.04 for k in range(100)]
    #grid_A = np.linspace(-2,2,100)
    
    def value_v2(x):
        return 1/gamma
    
    my_opt_strategy3 = []
    
    for t in range(len(Trading_Ret)-1,-1,-1):
        Interpol_Y2 = []; opt_stra_at_t2 = []
        for A in grid_A:
            def my_obj3(x):
                return -((1 + x*(mu+sigma) + (1-x)*R0 - lambda_*np.abs(x-A))**gamma * value_v2(x*(1+mu+sigma) / (1+x*(mu+sigma)+(1-x)*R0))
                         + (1 + x*(mu-sigma) + (1-x)*R0 - lambda_*np.abs(x-A))**gamma * value_v2(x*(1+mu-sigma) / (1+x*(mu-sigma)+(1-x)*R0))) / 2
            
            opt3 = scipy.optimize.minimize_scalar(my_obj3)
            
            Interpol_Y2.append(((1 + opt3.x*(mu+sigma) + (1-opt3.x)*R0 - lambda_*np.abs(opt3.x-A))**gamma * value_v2(opt3.x*(1+mu+sigma) / (1+opt3.x*(mu+sigma)+(1-opt3.x)*R0))
                               + (1 + opt3.x*(mu-sigma) + (1-opt3.x)*R0 - lambda_*np.abs(opt3.x-A))**gamma * value_v2(opt3.x*(1+mu-sigma) / (1+opt3.x*(mu-sigma)+(1-opt3.x)*R0))) / 2)
            
            opt_stra_at_t2.append(opt3.x)    
        
        value_v2 = InterpolatedUnivariateSpline(grid_A, Interpol_Y2, None, [grid_A[0], grid_A[-1]], 1)
        my_opt_strategy3.append(opt_stra_at_t2)
    
    #if i==0:
        #print(my_opt_strategy3)
    #print('\n')
    my_opt_strategy3 = np.array(my_opt_strategy3)
    
    #now computing the PnL forwardly
    if i == 0:
        temp = 1
    else:
        temp = PnL3[-1]
    
    opt_A = 0
    for j in range(len(Trading_Ret)):
        alpha3 = InterpolatedUnivariateSpline(grid_A, my_opt_strategy3[len(Trading_Ret)-j-1,:], None, [grid_A[0], grid_A[-1]], 1)
        opt_alpha3 = alpha3(opt_A)
        #print(opt_alpha3)
        temp *= 1 + (1-opt_alpha3)*R0 + opt_alpha3*Trading_Ret[j] - lambda_*np.abs(opt_alpha3 - opt_A)
        if j < len(Trading_Ret)-1:
            opt_A = opt_alpha3*(1+Trading_Ret[j+1]) / (1+opt_alpha3*Trading_Ret[j+1]+(1-opt_alpha3)*R0)
        
    for j in range(len(Trading_Ret)):
        PnL3.append(temp)
        
    print(i)

0


D:\APPLICATIONS\Anaconda\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


1
2
3
4
5
6
7


D:\APPLICATIONS\Anaconda\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars


8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [112]:
#(c) continuing
#save data
np.savetxt("PnL_in_c.csv",PnL3,fmt='%1s',delimiter=',')

#output
print("Annualized mean and variance: ")
print(np.mean(PnL3)*AnnualizedDay, np.var(PnL3)*AnnualizedDay)
print('\n')
print("Sharpe Ratio is: ")
print(np.mean(PnL3)*AnnualizedDay / np.sqrt(np.var(PnL3)*AnnualizedDay))

Annualized mean and variance: 
0.28153354173492134 0.006363812582328052


Sharpe Ratio is: 
3.5291608544159025
